In [3]:
import numpy as np
import pandas as pd


In [4]:
mpstDF= pd.read_csv("mpst.csv")
mpstDF

,imdb_id,title,plot_synopsis,tags,split,synopsis_source
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"cult, horror, gothic, murder, atmospheric",train,imdb
1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",violence,train,imdb
2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",romantic,test,imdb
3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","inspiring, romantic, stupid, feel-good",train,imdb
4,tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",val,imdb
...,...,...,...,...,...,...
14823,tt0219952,Lucky Numbers,"In 1988 Russ Richards (John Travolta), the wea...","comedy, murder",test,wikipedia
14824,tt1371159,Iron Man 2,"In Russia, the media covers Tony Stark's discl...","good versus evil, violence",train,wikipedia
14825,tt0063443,Play Dirty,During the North African Campaign in World War...,anti war,train,wikipedia
14826,tt0039464,High Wall,Steven Kenet catches his unfaithful wife in th...,murder,test,wikipedia


In [5]:
final_mpstDF_train=mpstDF[mpstDF["split"]=="train"]
final_mpstDF_validation=mpstDF[mpstDF["split"]=="val"]
final_mpstDF_test=mpstDF[mpstDF["split"]=="test"]
print("Shape Train",final_mpstDF_train.shape)
print("Shape Validation",final_mpstDF_validation.shape)
print("Shape Test",final_mpstDF_test.shape)

Shape Train (9489, 6)
Shape Validation (2373, 6)
Shape Test (2966, 6)


In [15]:
final_mpstDF_train.index

Int64Index([    0,     1,     3,     6,     7,     8,     9,    10,    11,
               12,
            ...
            14810, 14811, 14813, 14815, 14817, 14818, 14820, 14822, 14824,
            14825],
           dtype=&#39;int64&#39;, length=9489)

In [8]:
final_mpstDF_validation.index

Int64Index([    0,     1,     3,     6,     7,     8,     9,    10,    11,
               12,
            ...
            14810, 14811, 14813, 14815, 14817, 14818, 14820, 14822, 14824,
            14825],
           dtype=&#39;int64&#39;, length=9489)

In [9]:
final_mpstDF_test.index

Int64Index([    2,    15,    19,    24,    27,    31,    35,    37,    41,
               42,
            ...
            14776, 14801, 14806, 14808, 14814, 14816, 14819, 14823, 14826,
            14827],
           dtype=&#39;int64&#39;, length=2966)

In [10]:
#READING THE BERT EMBEDDINGS AND Y MATRIX
bert_embedding=np.load("D:\CodeRepo\Thesis\Thesis\BERT\embeddings.npz")
label_values=np.load("D:\CodeRepo\Thesis\Thesis\EDA\Y.npz")

In [11]:
type1_BERT_Embeddings=bert_embedding["t1"]
type2_BERT_Embeddings=bert_embedding["t2"]
label_values=label_values["arr_0"]

In [12]:
type1_BERT_Embeddings.shape

(14828, 768)

In [13]:
label_values.shape

(14828, 71)

In [20]:
def get_partition_Embeddings(x_t1,x_t2,y,df,partition_nm):
    _df=df[df["split"]==partition_nm]
    index_list=list(_df.index)
    temp_array_x_t1=[]
    temp_array_x_t2=[]
    temp_array_y=[]
    for index in index_list:
        temp_array_x_t1.append(x_t1[index,:])
        temp_array_x_t2.append(x_t2[index,:])
        temp_array_y.append(y[index,:])
    temp_array_x_t1=np.array(temp_array_x_t1)
    temp_array_x_t2=np.array(temp_array_x_t2)
    temp_array_y=np.array(temp_array_y)
    return temp_array_x_t1,temp_array_x_t2, temp_array_y

In [23]:
# FOR TRAIN
type1_BERT_Embeddings_Train,type2_BERT_Embeddings_Train,label_values_Train=get_partition_Embeddings(type1_BERT_Embeddings,type2_BERT_Embeddings,label_values,mpstDF,"train")
# FOR VALIDATION
type1_BERT_Embeddings_Val,type2_BERT_Embeddings_Val,label_values_Val=get_partition_Embeddings(type1_BERT_Embeddings,type2_BERT_Embeddings,label_values,mpstDF,"val")
# FOR TEST
type1_BERT_Embeddings_Test,type2_BERT_Embeddings_Test,label_values_Test=get_partition_Embeddings(type1_BERT_Embeddings,type2_BERT_Embeddings,label_values,mpstDF,"test")

In [26]:
# SEPARTAING THE TRAIN TEST AND VALIDATION ROWS
print(type1_BERT_Embeddings_Train.shape)
print(type2_BERT_Embeddings_Train.shape)
print(type1_BERT_Embeddings_Val.shape)
print(type2_BERT_Embeddings_Val.shape)
print(type1_BERT_Embeddings_Test.shape)
print(type2_BERT_Embeddings_Test.shape)
print(label_values_Train.shape)
print(label_values_Val.shape)
print(label_values_Test.shape)


(9489, 768)
(9489, 768)
(2373, 768)
(2373, 768)
(2966, 768)
(2966, 768)
(9489, 71)
(2373, 71)
(2966, 71)


In [ ]:
def _sigmoid(x):
    return 1. / (1. + np.exp(-x))
class ELM_MultiLabel:
    def __init__(self, input_nodes, hidden_nodes, output_nodes):
        self.__input_nodes=input_nodes
        self.__hidden_nodes=hidden_nodes
        self.__output_nodes=output_nodes
        #Type 1
        self.__beta = np.random.uniform(-1.,1.,size=(self.__hidden_nodes, self.__output_nodes))
        self.__alpha = np.random.uniform(-1.,1.,size=(self.__input_nodes, self.__hidden_nodes))
        self.__bias = np.zeros(shape=(self.__n_hidden_nodes,))
        #Type 2
        # self.__beta = np.random.normal(-1.,1.,size=(self.__hidden_nodes, self.__output_nodes))
        # self.__alpha = np.random.normal(-1.,1.,size=(self.__input_nodes, self.__hidden_nodes))
        # self.__bias = np.random.normal(shape=(self.__n_hidden_nodes,)) 
        self.__activation= lambda x: 1. / (1. + np.exp(-x)) # Sigmoid Function

    def fit(self, train_x, train_y, verbose=False):
        if verbose:
            print("train_x shape:", train_x.shape)
            print("train_y shape:", train_y.shape)
        # 1 Propagate data from Input to hidden Layer
        if verbose:
            print("Propagate data from Input to hidden Layer")
        inp= np.dot(train_x , self.__alpha)
        if verbose:
            print(inp)
            print("Adding Biases")
        inp = inp +  + self.__bias
        if verbose:
            print(inp)
            print("Applyin activation function")
        inp_activation= np.apply_along_axis(self.__activation, 1, inp)
        # This is the H matrix getting its Moore Penrose Inverse
        if verbose:
            print(inp_activation)
            print("Getting the Generalized Moore Penrose Inverse")
        generalizedInverse= np.linalg.pinv(inp_activation)
        if verbose:
            print(generalizedInverse)
            print("Finding Beta, output weights")
        # Now find output weight matrix Beta by
        self.__beta= np.dot(generalizedInverse,train_y)
    
    def predict(test):
        # the output will be computed for the input,
        # need to add the evaluation metrics as well

